# 批量parsing a2l/ulp

In [8]:
import sys
import os


# 遍历文件夹
def walkFile(file):
    for root, dirs, files in os.walk(file):

        # root 表示当前正在访问的文件夹路径
        # dirs 表示该文件夹下的子目录名list
        # files 表示该文件夹下的文件list
        for d in dirs:
            directory_files_in = os.path.join(root, d)
            print(os.listdir(directory_files_in))
        
        # 遍历文件
        for f in files:
            print("### " + os.path.join(root, f))
            
def list_all_files(directory):
    files_list = []
    for root, dirs, files in os.walk(directory):
        for f in files:
            file = os.path.join(root, f)
            if file.endswith("a2l") or file.endswith("ulp") or file.endswith("dat") or file.endswith("csv") or file.endswith("json"):
                if "(" in file or ")" in file:
                    file_ = file.replace("(", "")
                    file_ = file_.replace(")", "_")
                    os.rename(file, file_)
                files_list.append(file)
    return files_list


def check_file_prefix(prefix, file):
    if file.starswith(prefix):
        return True
    else:
        return False

def group_file_with_same_exp(file_list):
    result = {}
    for f in file_list:
        # get file prefix
        f_parts = f.split("/")
        f_prefix = "/".join(f_parts[0:-1])
        # add prefix key to result dict
        if f_prefix not in result:
            result[f_prefix] = []
        result[f_prefix].append(f)
    
    return result
        
def output_to_file(dict_, directory):
    for key in dict_:
        json_output = os.path.join(key, "parse_calibration.json")
        parts = key.split("/")
        file_name = parts[-1]
        path = os.path.join(directory, file_name)
        f = open(path, "w")
        files = dict_[key]
        l = ["", ""]
        for it in files:
            if it.endswith("a2l"):
                l[0] = it
            if it.endswith("ulp"):
                l[1] = it
        f.write(l[0])
        f.write("\n")
        f.write(l[1])
        f.write("\n")
        f.write(json_output)
        with open(json_output, "w") as f_out:
            pass
        f.close()
        
    
        print("echo    Parsing ...  ")
        java_cmd = "java -jar znbd-1.0-SNAPSHOT.jar  com.psagroup.calibrationparserapi.CalibrationParserApi2 %s %s %s > stderr 2>&1 "%(
            l[0], l[1], json_output
        )
        print(java_cmd)
        print("echo    Parsing OK ")
        
        
            


In [9]:
# file_dirs = '/Users/others2/Desktop/calibration/data/'

# a = list_all_files(file_dirs)
# b = group_file_with_same_exp(a)
# output_to_file(b, "/Users/others2/Desktop/output_ulp_json/")


# 重新组织map文件

In [126]:
import json
import numpy as np
import pandas as pd
from scipy import interpolate

# 先采集几个重要的标定量和采集量
map_file = "/Users/others2/Desktop/MAP.txt"
important_map = set()
sample_file = "/Users/others2/Desktop/sample.txt"
important_sample = set()
with open(map_file, "r") as f:
    for line in f:
        line = line.strip()
        important_map.add(line)
with open(sample_file, "r") as f:
    for line in f:
        line = line.strip()
        important_sample.add(line)
#important_map = {"AFC_SMOKE_LIM_A0_NORM_APM"}
print(important_map)

{'ACM_EGR_DMND_A0_NORM_APM', 'ITD_AFTER_SEP_SMOKE_NORM_APM', 'P_T_SCR_NH3_STORED_TARGET_APM', 'P_T_SCR_UREA_FLOW_LIMIT_APM', 'P_T_LNT_NOXR_ABORT_IMEP_MAX_APM', 'T_D_ENG_MAX_BRAKE_TORQUE_APM', 'AFC_SMOKE_LIM_A0_NORM_APM', 'FQD_AFTER_SMOKE_CONT_NORM_APM', 'T_D_MAX_FUEL_NORM_APM', 'T_D_PEDAL_POS_TO_TORQUE_0_APM', 'ITD_MAIN_TMNG_DMND_NORM_APM', 'PSE_INTERCOOLER_PRESS_DROP_APM', 'P_T_SCR_NH3_TARGET_LIMIT_APM', 'P_T_LNT_NOXR_RETRG_IMEP_MIN_APM', 'P_T_LNT_NOXR_TRG_IMEP_MIN_APM', 'P_T_SCR_EST_NH3_CORRECTION_APM', 'T_D_MAX_TORQ_COOL_TEMP_SCALE_APM', 'RPD_RAILP_DEMAND_NORM_APM', 'P_T_LNT_NOXR_TRG_DIMEP_APM', 'ACM_EGR_DMND_INERT_MIN_NORM_APM', 'P_T_LNT_NOXR_ABORT_IMEP_MIN_APM', 'ACM_VGT_BST_LIM_E_A0_NORM_APM', 'ITD_PILOT1_SEP_DMND_NORM_APM', 'P_T_SCR_NOX_NH3_RATIO_APM'}


In [127]:
# map stored in A2lMap class
class A2lMap():
    def __init__(self, name, desc, data, char_type):
        self.name = name
        self.desc = desc
        self.data = data
        self.char_type = char_type
        self.axisX = None
        self.axisY = None
        self.value = None
        self.get()
    
    def get(self):
        """
        解析MAP
        """
        self.axisX = [float(v) for v in self.data["axisX"]["value"]]
        self.inputqX = self.data["axisX"]["inputquantity"]
        self.unitX = self.data["axisX"]["unit"]
        
        self.axisY = [float(v) for v in self.data["axisY"]["value"]]
        self.inputqY = self.data["axisY"]["inputquantity"]
        self.unitY = self.data["axisY"]["unit"]
        
        x_size = len(self.axisX)
        y_size = len(self.axisY)
            
        readway = self.data["data"]["readway"]
        rows = [None for i in range(y_size)]
        if readway == "ROW_DIR":
            for idx in range(y_size):
                row = self.data["data"]["value"][idx*x_size : (idx+1)*x_size]
                rows[idx] = row  
            rows = np.array(rows).T
        self.value = rows
        
        row_number = self.value.shape[0]
        col_number = self.value.shape[1]
        
        assert isinstance(self.axisX, list), "axisX must be a list"
        assert isinstance(self.axisY, list), "axisY must be a list"
        assert len(self.axisX) == row_number, "axisX length not match"
        assert len(self.axisY) == col_number, "axisY length not match"
            
        
    def lookup(self, x, y):
        """
        根据标定量MAP的两个坐标轴的输入，通过双线性插值的方法获取对应的值
        """
        x_sz = len(self.axisX)
        y_sz = len(self.axisY)
        x_idx = 0
        y_idx = 0
        x_idx_before = x_idx
        y_idx_before = y_idx
        
        #print("x: %f, y: %f" % (x, y))
        # search axisX
        if x <= self.axisX[0]:
            x_idx = 0
        elif x>= self.axisX[x_sz-1]:
            x_idx = x_sz - 1
        else:
            while x >= self.axisX[x_idx] and x_idx < x_sz:
                x_idx += 1
        
        # search axisY
        if y <= self.axisY[0]:
            y_idx = 0
        elif y>= self.axisY[y_sz-1]:
            y_idx = y_sz - 1
        else:
            while y >= self.axisY[y_idx] and y_idx < y_sz:
                y_idx += 1
        
        #print("x_idx: %d, y_idx: %d" % (x_idx, y_idx))
        if x_idx == 0 or x_idx == x_sz - 1:
            x_idx_before = x_idx
        else:
            x_idx_before = x_idx - 1
        
        if y_idx == 0 or y_idx == y_sz - 1:
            y_idx_before = y_idx
        else:
            y_idx_before = y_idx - 1
        
        #print("x_idx_before: %d, y_idx_before: %d" % (x_idx_before, y_idx_before))
        
        Q11 = self.value[x_idx][y_idx_before]
        Q21 = self.value[x_idx][y_idx]
        Q12 = self.value[x_idx_before][y_idx_before]
        Q22 = self.value[x_idx_before][y_idx]
        
        #print("Q11: %f, Q21: %f, Q12: %f, Q22: %f"%(Q11, Q21, Q12, Q22))

        y_gap = self.axisY[y_idx] - self.axisY[y_idx_before]
        x_gap = self.axisX[x_idx] - self.axisX[x_idx_before]
        
        if x_gap == 0 and y_gap == 0:
            return Q11
        
        if x_gap == 0 and y_gap != 0:
            percent = (y - self.axisY[y_idx_before]) / y_gap 
            return (Q21 - Q11) * percent + Q11
        
        if y_gap == 0 and x_gap != 0:
            percent = (x - self.axisX[x_idx_before]) / x_gap
            return  (Q11 - Q12) * percent + Q12
        
        # Y 方向插值
        f1 = (y - self.axisY[y_idx_before]) / y_gap * (Q21 - Q11) + Q11
        f2 = (y - self.axisY[y_idx_before]) / y_gap * (Q22 - Q12) + Q12
        
        # X 方向插值
        fp = (x - self.axisX[x_idx_before]) / x_gap * (f1 - f2) + f2
        
        return fp
        
#         # Y 方向插值
#         f1 = (self.axisY[y_idx] - y) / y_gap * Q11 + (y - self.axisY[y_idx_before]) / y_gap * Q21
#         f2 = (self.axisY[y_idx] - y) / y_gap * Q12 + (y - self.axisY[y_idx_before]) / y_gap * Q22
        
#         #print("f1: %f, f2: %f"%(f1, f2))
        
#         # X 方向插值
#         fp = (self.axisX[x_idx] - x) / x_gap * f1 + (x - self.axisX[x_idx_before]) / x_gap * f2
        
#         return fp
            
    
# curve stored in A2lCurve class
class A2lCurve():
    def __init__(self, name, desc, data, char_type):
        """
        name: 标定量的名字
        desc: 标定量的描述
        data: 标定量的值
        """
        self.name = name
        self.desc = desc
        self.data = data
        self.char_type = char_type
        self.get()
        
    def get(self):
        self.axisX = self.data["axisX"]["value"]
        self.inputqX = self.data["axisX"]["inputquantity"]
        self.unitX = self.data["axisX"]["unit"]
        self.value = self.data["data"]["value"]
        
        assert isinstance(self.value, list), "value must be a list"
        assert len(self.value) == len(self.axisX), "axisX must be same length as value"
        
    def lookup(self, x):
        if x <= self.axisX[0]:
            return self.value[0]
    
        sz = len(self.axisX)
        if x >= self.axisX[sz-1]:
            return self.value[sz-1]
    
        i = 1
        while x<=self.axisX[i] and i<sz-1:
            i += 1
        gap = (x - self.axisX[i-1]) / (self.axisX[i] - self.axisX[i-1])
        
        return self.value[i-1] + gap * (self.value[i] - self.value[i-1])


        
        

In [133]:
def roll_to_1800s(file_directory):
    """
    1800s的数据展开，map curve 查值
    Parameters:
        file_directory: json文件和csv文件所在的目录
    Return:
        
    """
    files = list_all_files(file_directory)
    a2l_json_file = ""
    dat_csv = ""
    
    for name in files:
        if name.endswith("json"):
            a2l_json_file = name
        if name.endswith("csv"):
            dat_csv = name
        
    assert a2l_json_file != "", "a2l json is empty"
    assert dat_csv != "", "data csv is empty"
    
    # read csv to dataframe
    dat = pd.read_csv(dat_csv, sep=";")
    dat.drop([0, 1], inplace=True)   # 去掉第一行和第二行，第一行是量纲行，第二行有空字符
    dat.reset_index(drop=True, inplace=True)
    columns = dat.columns
    
    # read json to a dict
    js_dict = {}
    with open(a2l_json_file, "r") as f:
        for line in f:
            line = line.strip()
            js = json.loads(line)
            # 解析字段
            label = js["label"]         # 标定量
            if label not in important_map:
                continue
            char_type = js["charType"]  # 标定量的类型
            data = eval(js["data"])     # 标定量的值
            desc = js["description"]    # 标定量的描述
            if char_type == "MAP":
                a2l_map = A2lMap(label, desc, data, char_type)
                js_dict[label] = a2l_map
                
            elif char_type == "CURVE":
                a2l_curve = A2lCurve(label, desc, data, char_type)
                js_dict[label] = a2l_curve
    
    #a2l_AFC_SMOKE_LIM_A0_NORM_APM = js_dict["AFC_SMOKE_LIM_A0_NORM_APM"]
    #print(a2l_AFC_SMOKE_LIM_A0_NORM_APM.value)
    #print(a2l_AFC_SMOKE_LIM_A0_NORM_APM.axisX)
    #print(a2l_AFC_SMOKE_LIM_A0_NORM_APM.axisY)
    
    # 添加标定量到 csv
    for key in js_dict:
        dat[key] = 0.0
    
    map_labels = list(js_dict.keys())
    bad_label = []
    rows_cnt = dat.shape[0]
    for calib_var in map_labels:
        for index in range(rows_cnt):
            #print("line: %d"%(index))
            a2l_obj = js_dict[calib_var]
            label = a2l_obj.name
            char_type = a2l_obj.char_type
            
            #if label in ["PSE_INTERCOOLER_PRESS_DROP_APM", "P_T_SCR_NH3_TARGET_LIMIT_APM"]:
            #    continue
            
            try:
                if char_type == "CURVE":
                    x_inputquantity = a2l_obj.inputqX
                    x_inputquantity_value = float(dat.at[index, x_inputquantity])
                    lookup_value = a2l_obj.lookup(x_inputquantity_value)
                    dat.at[index, label] = lookup_value
                elif char_type == "MAP":
                    x_inputquantity = a2l_obj.inputqX
                    y_inputquantity = a2l_obj.inputqY
                    x_inputquantity_value = float(dat.at[index, x_inputquantity])
                    y_inputquantity_value = float(dat.at[index, y_inputquantity])
                    #print("x_inputquantity: %s, x_inputquantity_value:%f"%(x_inputquantity, x_inputquantity_value))
                    #print("y_inputquantity: %s, y_inputquantity_value:%f"%(y_inputquantity, y_inputquantity_value))
                    lookup_value = a2l_obj.lookup(x_inputquantity_value, y_inputquantity_value)
                    dat.at[index, label] = lookup_value
            except:
                if char_type == "CURVE":
                    bad_label.append("%s  %s"%(label, a2l_obj.inputqX))
                else:
                    bad_label.append("%s  %s  %s"%(label, a2l_obj.inputqX, a2l_obj.inputqY))
    bad_label = list(set(bad_label))
    print("bad_label: \n%s "%("\n".join(bad_label)))
    return dat

# test
join_dat = roll_to_1800s("/Users/others2/Desktop/calibration/data/20191029--1/")
join_dat.to_csv("/Users/others2/Desktop/xx.csv")

        

bad_label: 
P_T_SCR_NH3_TARGET_LIMIT_APM  P_T_Scr2_stored_nh3  TSE_Scr_bed_temp
PSE_INTERCOOLER_PRESS_DROP_APM  ACM_Compressor_flow_sel 


In [119]:
time_step = 7000
print(join_dat.at[time_step, "IN_Engine_cycle_speed"] )
print(join_dat.at[time_step, "AFC_Smoke_limit_dsrd_imep"])
print(join_dat.at[time_step, "AFC_SMOKE_LIM_A0_NORM_APM"])

2315.75
463.5
28.2471484375


In [110]:
(2000.0 - 1800) / (2000 - 1800) *(18.359 - 18.590) + 18.590

18.359

In [111]:
(2000.0 - 1800) / (2000 - 1800) * (19.430 - 19.699) + 19.699

19.43

In [112]:
(2315.75 - 2200) / (2400 - 2200) *(19.430 - 18.359) + 18.395

19.01484125

In [113]:
(19.01484125 - 18.978818359375) / 18.978818359375

0.001898057610483655

In [120]:
(463.5 - 500) / 100 * (27.965 - 28.738) + 27.738

28.020145

In [121]:
(463.5 - 500) / 100 * (27.395 - 28.168) + 28.168

28.450145

In [122]:
(3000 - 2800) / 200 *(28.020145 - 28.450145) + 28.450145

28.020145

In [123]:
(28.2471484375 - 28.020145) / 28.020145

0.008101436930465646